---

 📌 Call Transcript Search Agent (RULER-Based Evaluation)

In this notebook, you will use the **Agent Reinforcement Trainer) (ART)** framework to train an agent that answers questions about **banking call transcripts**.

The agent learns by performing tool-using rollouts and is scored with **RULER**, a robust relative ranking evaluator that helps the model learn to answer more accurately over time.

 🔧 Key Tools Available to the Agent

1. `search_inbox` — Search for calls using keywords.  
2. `read_call` — Read the full transcript of a specific call by ID.  
3. `return_final_answer` — Return the final answer and the IDs of calls used.

 📆 Training Overview

You will train the model in small batches (groups) of scenarios. Each scenario has several stochastic rollouts, which RULER scores, and those scores inform the gradient updates.

ART will also periodically run **validation** rollouts and log the trajectories.

---

 🚀 Ready to Train?

> To train this agent, click **Runtime > Run all**.  
> Don’t forget to set your environment variables:  
> - `OPENAI_API_KEY`  
> - `HF_TOKEN` (for dataset access)  
> - `WANDB_API_KEY` (for logging, optional)

This notebook uses:
- **call transcripts** with multi-turn formats like `"Customer: ... | Agent: ..."`
- **RULER** for scoring training rollouts
- **ART’s training loop** with GRPO reinforcement learning

Let’s get started!

---

### Installation


In [1]:
!pip install pyarrow datasets
#uv would fail to install these in macOS due to arrow-cpp missing components

In [18]:
!uv pip install openpipe-art==0.5.2 langchain-core tenacity transformers peft accelerate safetensors huggingface_hub wandb

Using Python 3.11.13 environment at: /Users/venetispallikaras/opt/anaconda3/envs/python_uv_3_11_mk2
Resolved 101 packages in 939ms                                       
⠙ Preparing packages... (0/8)                                                   
⠙ Preparing packages... (0/8)-------------------     0 B/523.63 KiB          
⠙ Preparing packages... (0/8)-------------------     0 B/523.63 KiB          
mpmath               ------------------------------     0 B/523.63 KiB
⠙ Preparing packages... (0/8)-------------------     0 B/11.44 MiB           
mpmath               ------------------------------     0 B/523.63 KiB
⠙ Preparing packages... (0/8)-------------------     0 B/11.44 MiB           
mpmath               ------------------------------     0 B/523.63 KiB
⠙ Preparing packages... (0/8)-------------------     0 B/11.44 MiB           
mpmath               ------------------------------     0 B/523.63 KiB
sympy                ------------------------------     0 B/6.01 MiB
⠙ Prep

In [19]:
import os
import json
from typing import Dict, Any, List, Literal, Optional

from datasets import DatasetDict, Dataset, Features, Sequence, Value, load_dataset
from huggingface_hub import HfApi
from openai import OpenAI
from tqdm import tqdm


import random
from dataclasses import dataclass, asdict
from textwrap import dedent
from pydantic import BaseModel, Field

import sqlite3
from datetime import datetime

import logging
import json
from textwrap import dedent

from openai import AsyncOpenAI
from litellm import acompletion
import weave
from langchain_core.utils.function_calling import convert_to_openai_tool
from tenacity import retry, stop_after_attempt

from pydantic import BaseModel, Field

import art
from art.utils.strip_logprobs import strip_logprobs

from dotenv import load_dotenv

import art
from art.serverless.backend import ServerlessBackend

import os, re, shutil, tempfile
import wandb
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel


load_dotenv()

random.seed(117)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/venetispallikaras/opt/anaconda3/envs/python_uv_3_11_mk2/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/venetispallikaras/opt/anaconda3/envs/python_uv_3_11_mk2/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/venetispallikaras/opt/anaconda3/envs/python_uv_3_11_m

<a name="Environment-Variables"></a>

### Environment Variables

Later on in the notebook, we'll be creating a model that can automatically logs metrics to Weights & Biases and chat completions to Weave. We'll also be using W&B Training for training and inference. In order to do so, you'll need to provide your Weights & Biases API key as an environment variable.

**OpenAI (used for RULER judge model)**

Our RULER reward function queries third-party models to judge the quality of the agent's performance. Any model supported by LiteLLM works. For this example we'll use OpenAI's o4-mini model, so we'll need to set the `OPENAI_API_KEY` environment variable.

In [ ]:
# Required for RULER judge model
os.environ["OPENAI_API_KEY"] = ""

# Required for Weights & Biases
os.environ["WANDB_API_KEY"] = ""

# Required for Hugging Face Hub
os.environ["HF_TOKEN"] = ""

if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY is required for RULER functionality when using openai/o4-mini."
    )

if not os.environ.get("WANDB_API_KEY"):
    raise ValueError("WANDB_API_KEY is required for inference, training, and logging to Weights & Biases.")

if not os.environ.get("HF_TOKEN"):
    raise ValueError("HF_TOKEN is required for uploading datasets to the Hugging Face Hub.")


### Dataset Generation

We utilize OpenAI models in order to create a dataset for our task. The dataset consists of transcripts as well as questions and answer pairs. The question is what the user asks(ISSUES AND COMPLAINTS WITH THEIR BANK PRODUCTS), e.g. what are the credit issues the customer faced. The answer is essentially the LLM reply, where it identifies the issues and complaints and provides context.

In [7]:

# ##  Synthetic Dataset Generation using OpenAI LLM and Upload to Hugging Face with QNA pairs

# # ==== CONFIG ====
# TARGET_REPO_ID = "venetis/banking_calls_qna"  # change if you like
# NUM_EXAMPLES = 1000                           # how many synthetic calls
# MODEL_NAME = "gpt-4.1-mini"                   # cost-effective, adjust if needed


# client = OpenAI()
# api = HfApi()


# def build_generation_prompt() -> str:
#     return """
# You are generating synthetic banking call-center data.

# For ONE example, do the following:

# 1. Invent a realistic phone call between:
#    - a CUSTOMER (member of a bank or credit union)
#    - a SUPPORT AGENT

# 2. The main topic of the call must be a banking PRODUCT ISSUE, such as:
#    - CREDIT CARD (declined transactions, fraud alerts, wrong fees, interest rate, limit issues, replacement card, etc.)
#    - DEBIT CARD (declines, ATM issues, lost card, etc.)
#    - CHECKING or SAVINGS ACCOUNT (fees, overdraft, holds, missing deposits, etc.)
#    - LOAN or MORTGAGE (payment issues, wrong amount, interest confusion, payoff questions, etc.)

# 3. Structure of the call:
#    - At least 8 turns, at most 20 turns total.
#    - Alternate between CUSTOMER and AGENT.
#    - Around 10%–30% of the call should be small talk or off-topic (e.g., weather, "how are you", minor chit-chat).
#    - The rest should be focused on the main PRODUCT ISSUE and how it is investigated or resolved.
#    - The issue should be clearly understandable from the call.

# 4. Formatting of the transcript:
#    - Each turn must be formatted as: "Customer: ..."
#      or "Agent: ..."
#    - TURNS MUST BE SEPARATED BY THE PIPE CHARACTER " | ".
#      Example: "Customer: ... | Agent: ... | Customer: ..."
#    - Do NOT include any newline characters in the transcript.
#    - Do NOT include the "|" character inside the spoken text itself,
#      only as the separator between turns.

# 5. Question and answer:
#    - Based on the transcript, write ONE natural-language QUESTION someone might ask
#      about the PRODUCT ISSUE in the call. Examples:
#        - "What were the customer's credit card complaints?"
#        - "Why was the member's debit card repeatedly declined?"
#        - "What problem did the member report with their mortgage payment and how was it resolved?"
#    - The QUESTION must be answerable solely from the transcript.
#    - Write a short, factual ANSWER that directly answers the question,
#      summarizing the relevant parts of the call.

# 6. Output format:
#    Return ONLY a single JSON object with EXACTLY these keys:
#    - "question": string
#    - "answer"  : string
#    - "transcript": string

# Example of the REQUIRED JSON SHAPE (NOTE: This is just a SHAPE example, not content):

# {
#   "question": "What issue did the customer have with their credit card?",
#   "answer": "Their credit card was repeatedly declined due to a fraud alert.",
#   "transcript": "Customer: ... | Agent: ... | Customer: ..."
# }

# Now generate ONE such JSON object.
# """.strip()

# def generate_one_example() -> Dict[str, Any]:
#     """Generate one (question, answer, transcript) triple via the LLM."""
#     prompt = build_generation_prompt()

#     resp = client.chat.completions.create(
#         model=MODEL_NAME,
#         temperature=0.5,
#         messages=[
#             {
#                 "role": "system",
#                 "content": "You are a careful data generator. You ALWAYS return valid JSON with the exact fields requested.",
#             },
#             {"role": "user", "content": prompt},
#         ],
#     )

#     content = resp.choices[0].message.content

#     # Try to parse as JSON; if it fails, we skip this example.
#     try:
#         data = json.loads(content)
#     except json.JSONDecodeError:
#         print("⚠️ JSON parse error, skipping one example.")
#         return {}

#     # Basic validation
#     q = data.get("question", "").strip()
#     a = data.get("answer", "").strip()
#     t = data.get("transcript", "").strip()

#     if not q or not a or not t:
#         return {}

#     # Sanity check: ensure pipe-separated turns
#     if "|" not in t:
#         print("⚠️ No '|' in transcript, skipping.")
#         return {}

#     return {
#         "question": q,
#         "answer": a,
#         "transcript": t,
#     }

# def generate_dataset(num_examples: int) -> Dataset:
#     rows: List[Dict[str, Any]] = []

#     for _ in tqdm(range(num_examples), desc="Generating synthetic calls"):
#         ex = generate_one_example()
#         if ex:
#             rows.append(ex)

#     print(f"Generated {len(rows)} valid examples out of {num_examples} attempts.")

#     if not rows:
#         raise RuntimeError("No valid examples generated. Check prompts / API keys.")

#     ds = Dataset.from_list(rows)

#     # Optional: add an integer ID
#     ds = ds.map(lambda x, idx: {"id": idx}, with_indices=True)

#     return ds


# ds = generate_dataset(NUM_EXAMPLES)

# def push_to_hf(ds: Dataset, repo_id: str):
#     # Split into train/test if you like
#     ds_train = ds.select(range(int(0.9 * len(ds))))
#     ds_test  = ds.select(range(int(0.9 * len(ds)), len(ds)))

#     dsd = DatasetDict({"train": ds_train, "test": ds_test})

#     # You must be logged in or have HF_TOKEN set
#     dsd.push_to_hub(repo_id)
#     print(f"Pushed dataset to: {repo_id}")


# push_to_hf(ds, TARGET_REPO_ID)


In [8]:
# from datasets import load_dataset, DatasetDict

# # Your current dataset with only a "train" split
# REPO_ID = "venetis/banking_calls_qna"   
# # Load the single split
# ds_full = load_dataset(REPO_ID, split="train")

# # Shuffle for randomness
# ds_full = ds_full.shuffle(seed=42)

# # 90% train, 10% test
# n = len(ds_full)
# train_size = int(0.9 * n)

# ds_train = ds_full.select(range(train_size))
# ds_test  = ds_full.select(range(train_size, n))

# print("Train size:", len(ds_train))
# print("Test size:", len(ds_test))

# # Create split dict and push back to HF
# ds_dict = DatasetDict({"train": ds_train, "test": ds_test})
# ds_dict.push_to_hub(REPO_ID)

# print(f"✅ Updated {REPO_ID} with train + test splits.")

{"id": ..., 

"question": ..., 

"answer": ..., 

"transcript": ...}

where transcript is a multi-turn call like
Customer: ... | Agent: ... | Customer: ...

<a name="Environment"></a>

### Call Search Environment

ART allows your agent to learn by interacting with its environment.  
In this example, we create an environment where the agent can **search through call transcripts and answer questions about them**.

The agent will have access to three tools:

1. `search_calls` – Search for calls by keywords in their transcript using a full-text index.
2. `read_call` – Read a specific call by its call ID (returns the full multi-turn transcript).
3. `return_final_answer` – Return the final answer along with the source call IDs that were used.

The questions will typically be about **banking products and issues**, e.g.:

> *“What were the credit card complaints?”*  
> *“Why was the member’s debit card repeatedly declined?”*

The agent must learn to:

- translate such questions into the right **search queries** over the call transcripts,
- read the retrieved calls via `read_call`,
- and produce a concise, grounded answer via `return_final_answer`.

In [9]:
# ============================================================
# Call and Scenario data models
# ============================================================
max_results = 20 # max search results to return #TODO


class Call(BaseModel):
    """
    Representation of a call in our local SQLite database.

    NOTE:
    - Our HF dataset only has an ID and a transcript (multi-turn dialog).
    """
    call_id: str                 # string ID for the call (derived from `id` in HF)
    transcript: str              # full multi-turn text: "Customer: ... | Agent: ... | ..."


class Scenario(BaseModel):
    """
    Scenario used for training / evaluation, one row per HF example.

    Our HF dataset schema is:

        {
          "id": int,
          "question": str,
          "answer": str,
          "transcript": str
        }

    We add a `split` field when loading (train / test)
    """
    id: int
    question: str
    answer: str
    transcript: str
    split: Literal["train", "test"]  # added when we call load_scenarios


@dataclass
class SearchResult:
    """
    Search result for DB lookups.
    """
    message_id: str   # = call_id in the calls table
    snippet: str      # short excerpt from transcript


class FinalAnswer(BaseModel):
    """
    Final answer object used later by the agent (same shape as original).
    """
    answer: str
    source_ids: list[str]


# ============================================================
# Database configuration
# ============================================================

# Local SQLite DB with all call transcripts (for FTS search)
DB_PATH = "./calls.db"
CALL_DATASET_REPO_ID = "venetis/banking_calls_qna"

# Global database connection, reused across calls
db_conn = None


# ============================================================
# Database creation: from HF calls → SQLite + FTS
# ============================================================

def create_calls_database():
    """
    Create the call database from the Hugging Face dataset.

    - One table: calls(id, call_id, transcript)
    - One FTS5 virtual table: calls_fts(transcript) with triggers
      to keep it in sync with calls.

    We index ALL calls (no limit), deduplicating on (call_id, transcript).
    """
    print("Creating calls database from Hugging Face dataset...")
    print(
        "This will download and process the full call dataset. "
        "It may take a few minutes the first time..."
    )

    # -----------------------------
    # Database schema
    # -----------------------------
    SQL_CREATE_TABLES = """
    DROP TABLE IF EXISTS calls;
    DROP TABLE IF EXISTS calls_fts;

    CREATE TABLE calls (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        call_id TEXT UNIQUE,
        transcript TEXT
    );
    """

    SQL_CREATE_INDEXES_TRIGGERS = """
    CREATE INDEX idx_calls_call_id ON calls(call_id);

    CREATE VIRTUAL TABLE calls_fts USING fts5(
        transcript,
        content='calls',
        content_rowid='id'
    );

    CREATE TRIGGER calls_ai AFTER INSERT ON calls BEGIN
        INSERT INTO calls_fts (rowid, transcript)
        VALUES (new.id, new.transcript);
    END;

    CREATE TRIGGER calls_ad AFTER DELETE ON calls BEGIN
        DELETE FROM calls_fts WHERE rowid=old.id;
    END;

    CREATE TRIGGER calls_au AFTER UPDATE ON calls BEGIN
        UPDATE calls_fts SET transcript=new.transcript WHERE rowid=old.id;
    END;
    """

    # -----------------------------
    # Create empty DB
    # -----------------------------
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.executescript(SQL_CREATE_TABLES)
    conn.commit()

    # -----------------------------
    # Load HF dataset
    # -----------------------------
    print("Loading call dataset from Hugging Face...")

    # We only really need 'id' and 'transcript', but we define expected features
    # to make things explicit and robust to types.
    expected_features = Features(
        {
            "id": Value("int64"),
            "question": Value("string"),
            "answer": Value("string"),
            "transcript": Value("string"),
        }
    )

    dataset = load_dataset(
        CALL_DATASET_REPO_ID,
        features=expected_features,
        split="train",
    )

    print(f"Dataset contains {len(dataset)} total call examples")

    # -----------------------------
    # Populate DB with all calls
    # -----------------------------
    print("Populating database with call transcripts...")
    conn.execute("PRAGMA synchronous = OFF;")
    conn.execute("PRAGMA journal_mode = MEMORY;")
    conn.execute("BEGIN TRANSACTION;")

    record_count = 0
    skipped_count = 0
    duplicate_count = 0

    # Track (call_id, transcript) to avoid duplicates
    processed_calls = set()

    for call_data in tqdm(dataset, desc="Inserting calls"):
        # HF "id" → string call_id; transcript is required
        raw_id = call_data["id"]
        call_id = str(raw_id)
        transcript = call_data["transcript"]

        if not transcript or not transcript.strip():
            skipped_count += 1
            continue

        # Optional: filter out absurdly long transcripts 
        if len(transcript) > 20000:
            skipped_count += 1
            continue

        call_key = (call_id, transcript)
        if call_key in processed_calls:
            duplicate_count += 1
            continue
        processed_calls.add(call_key)

        cursor.execute(
            """
            INSERT INTO calls (call_id, transcript)
            VALUES (?, ?)
        """,
            (call_id, transcript),
        )

        record_count += 1

    conn.commit()

    # -----------------------------
    # Create FTS index and rebuild
    # -----------------------------
    print("Creating FTS index and triggers...")
    cursor.executescript(SQL_CREATE_INDEXES_TRIGGERS)
    cursor.execute('INSERT INTO calls_fts(calls_fts) VALUES("rebuild")')
    conn.commit()

    print(f"Successfully created calls database with {record_count} rows.")
    print(f"Skipped {skipped_count} rows (empty/too long).")
    print(f"Skipped {duplicate_count} duplicate calls.")
    return conn


def get_db_connection():
    """
    Get a (global) database connection.

    - If DB already exists, open it.
    - If not, build it from the HF dataset.
    """
    global db_conn
    if db_conn is None:
        if os.path.exists(DB_PATH):
            print(f"Loading existing call database from {DB_PATH}")
            db_conn = sqlite3.connect(DB_PATH, check_same_thread=False)
        else:
            db_conn = create_calls_database()
    return db_conn


# ============================================================
# Search & read helpers (used later by the agent tools)
# ============================================================

def search_calls(
    keywords: List[str],
    max_results: int = 10,
) -> List[SearchResult]:
    """
    Search the call database based on keywords in the transcript.

    """
    conn = get_db_connection()
    cursor = conn.cursor()

    where_clauses: List[str] = []
    params: List[str | int] = []

    if not keywords:
        raise ValueError("No keywords provided for search.")

    if max_results > max_results:
        raise ValueError(f"max_results must be less than or equal to {max_results}.")

    # FTS5 default is AND between terms.
    # We escape quotes and wrap each keyword in quotes for safety.
    fts_query = " ".join(f""" "{k.replace('"', '""')}" """ for k in keywords)
    where_clauses.append("fts.calls_fts MATCH ?")
    params.append(fts_query)

    sql = f"""
        SELECT
            c.call_id,
            snippet(calls_fts, 0, '<b>', '</b>', ' ... ', 15) as snippet
        FROM
            calls c JOIN calls_fts fts ON c.id = fts.rowid
        WHERE
            {" AND ".join(where_clauses)}
        LIMIT ?;
    """
    params.append(max_results)

    cursor.execute(sql, params)
    results = cursor.fetchall()

    # We reuse the field name `message_id` in SearchResult for compatibility
    return [SearchResult(message_id=row[0], snippet=row[1]) for row in results]


def read_call(call_id: str) -> Optional[Call]:
    """
    Retrieve a single call by its call_id from the DB.
    This is the analog of read_email.
    """
    conn = get_db_connection()
    cursor = conn.cursor()

    cursor.execute(
        "SELECT call_id, transcript FROM calls WHERE call_id = ?",
        (call_id,),
    )
    row = cursor.fetchone()

    if not row:
        return None

    call_id, transcript = row
    return Call(call_id=call_id, transcript=transcript)


# ============================================================
# Scenario loader (from HF Q&A dataset)
# ============================================================

def load_scenarios(
    split: Literal["train", "test"] = "train",
    limit: Optional[int] = None,
    shuffle: bool = True,
    seed: Optional[int] = None,
) -> List[Scenario]:
    """
    Load scenarios from the Hugging Face Q&A dataset.

    HF SCHEMA:
        {
          "id": int,
          "question": str,
          "answer": str,
          "transcript": str
        }

    """
    print(f"Loading {split} scenarios from Hugging Face...")
    dataset: Dataset = load_dataset(CALL_DATASET_REPO_ID, split=split)

    if shuffle or (seed is not None):
        if seed is not None:
            dataset = dataset.shuffle(seed=seed)
        else:
            dataset = dataset.shuffle()

    # Convert each row to a Scenario object
    scenarios = [
        Scenario(
            id=int(row["id"]),
            question=str(row["question"]),
            answer=str(row["answer"]),
            transcript=str(row["transcript"]),
            split=split,
        )
        for row in dataset
    ]

    if shuffle:
        if seed is not None:
            rng = random.Random(seed)
            rng.shuffle(scenarios)
        else:
            random.shuffle(scenarios)

    if limit is not None:
        scenarios = scenarios[:limit]

    print(f"Loaded {len(scenarios)} {split} scenarios.")
    return scenarios


# ============================================================
# Load training & validation scenarios and print a sample
# ============================================================

training_scenarios = load_scenarios(
    split="train",
    # limit=50,           
    shuffle=True,
    seed=117,
)

validation_scenarios = load_scenarios(
    split="test",
    # limit=20,
    shuffle=True,
    seed=117,
)

print("Call search environment created with full call dataset!")
print(
    f"Database contains the complete call dataset, "
    f"loaded {len(training_scenarios)} training scenarios and "
    f"{len(validation_scenarios)} validation scenarios."
)

# Print first scenario to sanity-check everything
print("\nSample scenario")
print("id:", training_scenarios[0].id)
print("question:", training_scenarios[0].question)
print("answer:", training_scenarios[0].answer)
print("transcript (first 300 chars):", training_scenarios[0].transcript[:300] + "...")
print("split:", training_scenarios[0].split)

Loading train scenarios from Hugging Face...
Loaded 729 train scenarios.
Loading test scenarios from Hugging Face...
Loaded 81 test scenarios.
Call search environment created with full call dataset!
Database contains the complete call dataset, loaded 729 training scenarios and 81 validation scenarios.

Sample scenario
id: 462
question: What issue did the customer report with their checking account and how was it resolved?
answer: The customer reported an unexpected overdraft fee on their checking account despite having a deposit that should have covered the balance. The agent investigated and found that the deposit was on hold for verification, which caused the overdraft. The agent explained the hold policy and removed the overdraft fee as a one-time courtesy.
transcript (first 300 chars): Customer: Hi, I noticed a $35 overdraft fee on my checking account and I don’t understand why. | Agent: Hello! I’m sorry to hear that. I’m happy to look into this for you. Before we start, how’s your

### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to search call transcripts effectively. We'll use a Qwen 3 14B model for this example.

In [10]:


# Declare the model
model = art.TrainableModel(
    project="call-transcript-search-agent",
    name="call-transcript-agent-whole-dataset-2",
    base_model="OpenPipe/Qwen3-14B-Instruct",
)

# ============================================================
# Initialize the Serverless backend
# ------------------------------------------------------------
# This tells ART to run both:
#   - inference (rollouts) and
#   - training (GRPO updates)
# on Weights & Biases' serverless infrastructure.
# ============================================================
backend = ServerlessBackend()

# ============================================================
# Register the model with the Serverless backend
# ------------------------------------------------------------
# This:
#   - creates an inference endpoint for the model
#   - sets up logging & telemetry
#   - prepares training resources for GRPO
#
# After this `await`, `model.inference_base_url` and
# `model.inference_api_key` are ready and used inside rollout().
# ============================================================
await model.register(backend)

print("✅ Registered TrainableModel")

✅ Registered TrainableModel


### Defining a Rollout

A rollout is a single episode of an agent performing its task. In this example,
the rollout function presents the agent with a **call search scenario**, and the
agent uses the available tools to:

1. Search through call transcripts (`search_inbox` / `search_calls`)
2. Read specific calls (`read_call_tool`)
3. Return a final answer with the call IDs used as evidence (`return_final_answer`)

When the agent provides a final answer, the `correct` metric is calculated based
on whether the answer matches the reference answer for that scenario.

In [ ]:

# -------------------------------------------------------------------
# Weave & logging setup 
# -------------------------------------------------------------------
logging.getLogger("weave").setLevel(logging.CRITICAL)

weave.init(
    model.project,
    settings={"print_call_link": False},
    # remove logprobs before recording in Weave
    global_postprocess_output=strip_logprobs
)

# Decrease the number of turns to speed up training
MAX_TURNS = 6





# -------------------------------------------------------------------
# 2. Trajectory + wrapper types for rollouts
# -------------------------------------------------------------------
class ProjectTrajectory(art.Trajectory):
    final_answer: FinalAnswer | None = None


class CallScenario(BaseModel):
    """
    Wrapper passed into rollout by the training loop.
    """
    step: int
    scenario: Scenario


# -------------------------------------------------------------------
# 3. Rollout: agent interacts with call-search environment
# -------------------------------------------------------------------
@weave.op
async def rollout(model: art.Model, call_scenario: CallScenario) -> ProjectTrajectory:
    scenario = call_scenario.scenario

    # Initialize an empty trajectory
    traj = ProjectTrajectory(
        reward=0.0,
        messages_and_choices=[],
        metadata={
            "scenario_id": scenario.id,
            "step": call_scenario.step,
        },
    )

    # System prompt: call-search agent
    system_prompt = dedent(
        f"""
        You are a call transcript search agent.

        You are given:
        - A user question about banking issues (credit cards, loans, accounts, etc.).
        - Tools that let you:
            1) search through a database of call transcripts by keywords, and
            2) read the full text of a specific call by its ID,
            3) return a final answer with the IDs of the calls you used.

        Each call transcript is a multi-turn conversation with this format:

            "Customer: ... | Agent: ... | Customer: ... | ..."

        Your job:
        1. Use the search tool to find calls relevant to the question.
        2. Use the read tool to inspect promising calls in detail.
        3. Identify which call(s) actually match the issue described in the question.
        4. Provide a concise, factual answer grounded in the transcripts.
        5. When you are done, call `return_final_answer` with:
           - your final answer, and
           - the list of call IDs you relied on.

        You may take up to {MAX_TURNS} turns, so if your first search is not good
        enough, refine your keywords and try again.
        """
    )

    # Seed the conversation with system + user question
    traj.messages_and_choices = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": scenario.question},
    ]

    # ----------------- Tool definitions -----------------

    def search_inbox(keywords: List[str]) -> list[dict]:
        """
        Search the calls database for transcripts matching the given keywords.

        Returns a list of dictionaries so the LLM can reason over them.
        Each dictionary will typically look like:
            { "message_id": "...", "snippet": "..." }
        """
        results = search_calls(
            keywords=keywords,
            max_results=max_results,
        )
        return [asdict(result) for result in results]

    def return_final_answer(
        answer: str, reference_message_ids: List[str]
    ) -> "FinalAnswer":
        """
        Return the final answer and the IDs of the calls that were used.

        This is the termination signal for the rollout.
        """
        return FinalAnswer(answer=answer, source_ids=reference_message_ids)

    # Collect tools; read_call is assumed to be defined in the Environment cell
    tools = [search_inbox, read_call, return_final_answer]
    tools_by_name = {t.__name__: t for t in tools}
    traj.tools = [convert_to_openai_tool(t) for t in tools]

    # Async client to query the current policy via ART's inference endpoint
    client = AsyncOpenAI(
        base_url=model.inference_base_url,
        api_key=model.inference_api_key,
    )

    # ----------------- Tool-calling loop -----------------
    for _ in range(MAX_TURNS):
        response = await client.chat.completions.create(
            model=model.get_inference_name(),
            temperature=0.7,
            messages=traj.messages(),
            tools=traj.tools,
        )

        response_message = response.choices[0].message
        traj.messages_and_choices.append(response.choices[0])

        # If the model didn't call any tools, trajectory ends here
        if not response_message.tool_calls:
            return traj

        try:
            for tool_call in response_message.tool_calls:
                tool_name: str = tool_call.function.name
                if tool_name in tools_by_name:
                    tool_args = json.loads(tool_call.function.arguments or "{}")
                    tool_to_call = tools_by_name[tool_name]
                    result = tool_to_call(**tool_args)

                    # Record tool output in the trajectory
                    traj.messages_and_choices.append(
                        {
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "name": tool_name,
                            "content": str(result),
                        }
                    )

                    # If the agent calls return_final_answer, we evaluate and stop
                    if tool_name == "return_final_answer":
                        traj.final_answer = result

                        return traj
        except Exception as e:
            print(f"Error executing tool call: {e}")
            return traj

    # If we reach here, the agent used up all turns without a final answer
    # Reward stays 0, but we still log the partial trajectory.
    return traj


print("✅ Rollout + granular correctness grading defined for call transcripts.")

weave: Logged in as Weights & Biases user: venetispall.
weave: View Weave data at https://wandb.ai/venetispall/call-transcript-search-agent/weave


✅ Rollout + granular correctness grading defined for call transcripts.


<a name="ruler"></a>

### How RULER works

**RULER** leverages two key insights:

1. Relative scoring is easier than absolute scoring: It's easier for an LLM to rank several solutions relative to each other than to score them in isolation
2. GRPO only needs relative scores: Since GRPO normalizes scores within each group, only the relative rankings matter, not absolute values

The process:

1. Generate N trajectories for a given scenario
2. Pass all N trajectories to **RULER**
3. **RULER** deduplicates common prefixes (e.g., identical system messages)
4. An LLM judge scores each trajectory from 0 to 1 based on goal achievement
5. These scores are used directly as rewards in GRPO training

To learn more about **RULER**, check out the [RULER docs](https://art.openpipe.ai/fundamentals/ruler).


In [12]:
# import art
# from art.rewards import ruler_score_group

# # Test RULER with a simple example
# base_messages = [
#     {"role": "system", "content": "You count numbers using numeric symbols."},
#     {"role": "user", "content": "Count to 10."},
# ]

# good_trajectory = art.Trajectory(
#     messages_and_choices=[
#         *base_messages,
#         {"role": "assistant", "content": "1, 2, 3, 4, 5, 6, 7, 8, 9, 10"},
#     ],
#     reward=0,
# )

# mediocre_trajectory = art.Trajectory(
#     messages_and_choices=[
#         *base_messages,
#         {
#             "role": "assistant",
#             "content": "one, two, three, four, five, six, seven, eight, nine, ten",
#         },
#     ],
#     reward=0,
# )

# bad_trajectory = art.Trajectory(
#     messages_and_choices=[
#         *base_messages,
#         {"role": "assistant", "content": "a, b, c, d, e, f, g, h, i, j"},
#     ],
#     reward=0,
# )

# sample_group = art.TrajectoryGroup(
#     trajectories=[
#         good_trajectory,
#         mediocre_trajectory,
#         bad_trajectory,
#     ]
# )

# judged_group = await ruler_score_group(sample_group, "openai/o4-mini", debug=True)
# assert judged_group is not None

# # Display rankings
# sorted_trajectories = sorted(
#     judged_group.trajectories, key=lambda t: t.reward, reverse=True
# )
# for rank, traj in enumerate(sorted_trajectories, 1):
#     messages = traj.messages()
#     print(f"\nRank {rank}: Score {traj.reward:.3f}")
#     print(f"  Response: {messages[-1]['content'][:50]}...")

<a name="Loop"></a>

### Training Loop

The training loop is where the magic happens. For each of the 'max_steps' steps or 'epochs' defined below, the rollout function will be called multiple times in parallel. Each scenario will produce a trajectory, which will be used to update the model.

The `gather` step will wait for all of the trajectories to be generated, then it will use RULER to assign relative scores to each trajectory.

Our notebook will then delete all but the most recent checkpoint and train the model on the scored trajectories.


In [14]:
# -------------------------------------------------------------------
# Training loop for call-transcript search agent (RULER-only evaluation)
# -------------------------------------------------------------------
from art.utils import iterate_dataset
from art.rewards import ruler_score_group
import math


training_config = {
    # A "group" = one scenario with multiple stochastic rollouts
    "groups_per_step": 3,
    "num_epochs": 2,
    "rollouts_per_group": 4,   # keep moderate to avoid backend overload
    "learning_rate": 1e-5,
    #"max_steps": 10,     #TODO      # keep small for demo; increase when stable
    "validation_step_interval": 25,
}

# Iterator over training scenarios (HF dataset -> Scenario objects)
training_iterator = iterate_dataset(
    training_scenarios,
    groups_per_step=training_config["groups_per_step"],
    num_epochs=training_config["num_epochs"],
    initial_step=await model.get_step(),
)
##########
# Count total training scenarios you will iterate over:
num_training = len(training_scenarios)

# Grab your config values
groups_per_step = training_config["groups_per_step"]
num_epochs = training_config["num_epochs"]

# Compute how many batches per epoch
steps_per_epoch = math.ceil(num_training / groups_per_step)

# Compute total planned training steps
total_expected_steps = steps_per_epoch * num_epochs

print(f"Training scenarios: {num_training}")
print(f"Groups per step:   {groups_per_step}")
print(f"Epochs:            {num_epochs}")
print(f"Steps per epoch:   {steps_per_epoch}")
print(f"Total planned steps: {total_expected_steps}")

##########

for batch in training_iterator:
    if batch.step % training_config["validation_step_interval"] == 0 and batch.step > 0:
        print(
            f"Training step {batch.step}, epoch {batch.epoch}, epoch step {batch.epoch_step}"
        )
        print(f"Batch contains {len(batch.items)} scenarios")
    # ----------------- 1. Create trajectory groups -----------------
    train_groups = []
    for scenario in batch.items:
        # IMPORTANT: pass a *dict* into CallScenario so Pydantic is happy
        call_scenario = CallScenario(
            step=batch.step,
            scenario=scenario.model_dump(),  # <- key fix vs previous version
        )

        # For each scenario, create a TrajectoryGroup of multiple rollouts
        # (same question, different stochastic generations).
        train_groups.append(
            art.TrajectoryGroup(
                (
                    rollout(model, call_scenario)
                    for _ in range(training_config["rollouts_per_group"])
                )
            )
        )

    # ----------------- 2. Gather trajectories from backend -----------------
    finished_train_groups = await art.gather_trajectory_groups(
        train_groups,
        pbar_desc="gather-train",
        max_exceptions=training_config["rollouts_per_group"] * len(batch.items),
    )

    # Filter out any empty groups (no successful trajectories)
    finished_train_groups = [
        g for g in finished_train_groups if len(g.trajectories) > 0
    ]
    if not finished_train_groups:
        print("⚠️ No finished training groups for this batch, skipping step.")
        continue

    # ----------------- 3. Apply RULER scoring within each group -----------------
    judged_groups = []
    for group in finished_train_groups:
        # RULER re-scores trajectories in the group relatively.
        # It uses the per-trajectory reward you set in `rollout` if present,
        # or falls back to its own scoring config.
        judged_group = await ruler_score_group(
            group,
            "openai/o4-mini",
            debug=False,  # debug=True can produce very large payloads #TODO
        )
        # Guard against any groups that may come back empty
        if len(judged_group.trajectories) > 0:
            judged_groups.append(judged_group)

    if not judged_groups:
        print("⚠️ No judged groups after RULER, skipping train() this step.")
        continue

    # ----------------- 4. Optional validation + logging -----------------
    if batch.step % training_config["validation_step_interval"] == 0:
        print("Running validation at step", batch.step)

        validation_groups = []
        for scenario in validation_scenarios:
            val_call_scenario = CallScenario(
                step=batch.step,
                scenario=scenario.model_dump(),
            )
            # For validation, one rollout per scenario is usually enough
            validation_groups.append(
                art.TrajectoryGroup(
                    [rollout(model, val_call_scenario)]
                )
            )

        finished_validation_groups = await art.gather_trajectory_groups(
            validation_groups,
            pbar_desc="gather-val",
            max_exceptions=len(validation_scenarios),
        )

        # Filter out empty validation groups
        finished_validation_groups = [
            g for g in finished_validation_groups if len(g.trajectories) > 0
        ]

        # Log validation trajectories; if logging fails (e.g. 422),
        # we print the error but do not crash training.
        if finished_validation_groups:
            try:
                await model.log(finished_validation_groups, split="val")
            except Exception as e:
                print(f"⚠️ Validation logging failed: {e}")

    # ----------------- 5. GRPO parameter update -----------------
    # Clean up older checkpoints to keep storage usage low
    await model.delete_checkpoints()

    # Train on the judged groups; if the backend has an internal error,
    # we surface it but don't kill the notebook.
    try:
        await model.train(
            judged_groups,
            config=art.TrainConfig(
                learning_rate=training_config["learning_rate"],
            ),
        )
    except Exception as e:
        print(f"⚠️ model.train failed with error: {e}")
        # Optionally `break` here if you want to stop on training errors
        break

    if batch.step % training_config["validation_step_interval"] == 0 and batch.step > 0:
        print(f"Completed training step {batch.step}")

    # ----------------- 6. Early stop for demo -----------------
    if "max_steps" in training_config and training_config.get("max_steps") is not None:
        print("Reached max_steps, stopping training loop.")
        break

print("✅ Finished GRPO training loop for call-transcript search agent (RULER).")

Training scenarios: 729
Groups per step:   3
Epochs:            2
Steps per epoch:   243
Total planned steps: 486


Iterating dataset:  66%|######6   | 321/486 [00:00<?, ?batch/s]

gather-train:   0%|          | 0/12 [00:00<?, ?it/s]

train: 0it [00:00, ?it/s]

gather-train:   0%|          | 0/12 [00:00<?, ?it/s]

...

gather-train:   0%|          | 0/12 [00:00<?, ?it/s]

train: 0it [00:00, ?it/s]

gather-train:   0%|          | 0/12 [00:00<?, ?it/s]

train: 0it [00:00, ?it/s]

✅ Finished GRPO training loop for call-transcript search agent (RULER).


### Using the Model

Just like that, you've trained an agent to search emails and answer questions! Now it's time to use your model outside of the training loop.

Check out the code below for a small demo of the model you just trained!


In [15]:
# Test the trained model using the rollout function
# This uses the same inference path as training (no special test logic)

print("Testing the trained model with a real call scenario...\n")

# Use a scenario from our training set
test_scenario = training_scenarios[0]

print(f"Test scenario ID: {test_scenario.id}")
print(f"Question: {test_scenario.question}")
print(f"Expected answer: {test_scenario.answer}")
print("-" * 50)

# Wrap the scenario exactly as expected by rollout
test_call_scenario = CallScenario(
    step=0,
    scenario=test_scenario,
)

# Run the rollout function with the trained model
result_trajectory = await rollout(model, test_call_scenario)

print("Agent's trajectory:")
print("-" * 20)

# Display the conversation (system, user, assistant, tools)
messages = result_trajectory.messages()
for i, msg in enumerate(messages):
    role = msg.get("role", "unknown")
    content = msg.get("content", "")
    tool_calls = msg.get("tool_calls", [])

    if role == "system":
        print(
            f"[SYSTEM]: {content[:100]}..."
            if len(content) > 100
            else f"[SYSTEM]: {content}"
        )
    elif role == "user":
        print(f"[USER]: {content}")
    elif role == "assistant":
        if tool_calls:
            print(f"[ASSISTANT - TOOL CALLS]: {tool_calls}")
        if content:
            print(f"[ASSISTANT]: {content}")
    elif role == "tool":
        tool_name = msg.get("name", "unknown_tool")
        print(
            f"[TOOL - {tool_name}]: {content[:200]}..."
            if len(content) > 200
            else f"[TOOL - {tool_name}]: {content}"
        )

    print()

print("-" * 50)

# Print final answer and evaluation
if result_trajectory.final_answer:
    print(f"Agent's Final Answer:\n{result_trajectory.final_answer.answer}")
    print(f"Source Call IDs Used: {result_trajectory.final_answer.source_ids}")
else:
    print("No final answer provided by the agent.")

print(f"\nExpected Answer:\n{test_scenario.answer}")

# Optional: show judge scores if available
if result_trajectory.metrics:
    print("\nEvaluation metrics:")
    for k, v in result_trajectory.metrics.items():
        print(f"  {k}: {v}")

print("\n🎉 Call transcript agent testing completed!")
print(
    "The agent was evaluated using the same rollout and judge logic as during training."
)

Testing the trained model with a real call scenario...

Test scenario ID: 462
Question: What issue did the customer report with their checking account and how was it resolved?
Expected answer: The customer reported an unexpected overdraft fee on their checking account despite having a deposit that should have covered the balance. The agent investigated and found that the deposit was on hold for verification, which caused the overdraft. The agent explained the hold policy and removed the overdraft fee as a one-time courtesy.
--------------------------------------------------
Agent's trajectory:
--------------------
[SYSTEM]: 
You are a call transcript search agent.

You are given:
- A user question about banking issues (cre...

[USER]: What issue did the customer report with their checking account and how was it resolved?

[ASSISTANT - TOOL CALLS]: [{'id': 'chatcmpl-tool-f8543628c90e41c285be5ae63d993dc4', 'type': 'function', 'function': {'name': 'search_inbox', 'arguments': '{"keywords"

In [16]:
import random

print("Testing the trained model with another call scenario...\n")

# Randomly sample a scenario (instead of always picking index 0)
test_scenario = random.choice(validation_scenarios)

print(f"Test scenario ID: {test_scenario.id}")
print(f"Question: {test_scenario.question}")
print(f"Expected answer: {test_scenario.answer}")
print("-" * 50)

# Wrap the scenario exactly as rollout expects
test_call_scenario = CallScenario(
    step=0,
    scenario=test_scenario,
)

# Run rollout (same inference path as training)
result_trajectory = await rollout(model, test_call_scenario)

print("Agent's trajectory:")
print("-" * 20)

# Display the full conversation
for msg in result_trajectory.messages():
    role = msg.get("role", "unknown")
    content = msg.get("content", "")
    tool_calls = msg.get("tool_calls", [])

    if role == "system":
        print(
            f"[SYSTEM]: {content[:120]}..."
            if len(content) > 120
            else f"[SYSTEM]: {content}"
        )
    elif role == "user":
        print(f"[USER]: {content}")
    elif role == "assistant":
        if tool_calls:
            print(f"[ASSISTANT - TOOL CALLS]: {tool_calls}")
        if content:
            print(f"[ASSISTANT]: {content}")
    elif role == "tool":
        tool_name = msg.get("name", "unknown_tool")
        print(
            f"[TOOL - {tool_name}]: {content[:300]}..."
            if len(content) > 300
            else f"[TOOL - {tool_name}]: {content}"
        )

    print()

print("-" * 50)

# Final answer summary
if result_trajectory.final_answer:
    print("Agent's Final Answer:")
    print(result_trajectory.final_answer.answer)
    print(f"Source Call IDs Used: {result_trajectory.final_answer.source_ids}")
else:
    print("⚠️ No final answer provided by the agent.")

print("\nExpected Answer:")
print(test_scenario.answer)

# Optional: show grading details
if result_trajectory.metrics:
    print("\nEvaluation metrics:")
    for k, v in result_trajectory.metrics.items():
        print(f"  {k}: {v}")

print("\n✅ Call transcript agent test completed.")
print(
    "This run used the same rollout, tools, and scoring logic as training "
    "to ensure behavior is representative of learned policy."
)

Testing the trained model with another call scenario...

Test scenario ID: 406
Question: What issue did the customer have with their debit card and how was it resolved?
Expected answer: The customer reported that their debit card was declined multiple times at ATMs despite having sufficient funds. The agent checked the account and found a temporary hold on the card for security reasons. After verifying the customer's identity and recent transactions, the agent removed the hold and confirmed the card was active again.
--------------------------------------------------
Agent's trajectory:
--------------------
[SYSTEM]: 
You are a call transcript search agent.

You are given:
- A user question about banking issues (credit cards, loans, ac...

[USER]: What issue did the customer have with their debit card and how was it resolved?

[ASSISTANT - TOOL CALLS]: [{'id': 'chatcmpl-tool-03d45724c01947cebd99f75ca3d3e576', 'type': 'function', 'function': {'name': 'search_inbox', 'arguments': '{"keyw

In [21]:
# Test the trained model using the rollout function
# This uses the same inference path as training (no special test logic)

print("Testing the trained model with a real call scenario...\n")

# Use a scenario from our training set
test_scenario = training_scenarios[0]

print(f"Test scenario ID: {test_scenario.id}")
print(f"Question: {test_scenario.question}")
print(f"Expected answer: {test_scenario.answer}")
print("-" * 50)

# Wrap the scenario exactly as expected by rollout
test_call_scenario = CallScenario(
    step=0,
    scenario=test_scenario,
)

# Run the rollout function with the trained model
result_trajectory = await rollout(model, test_call_scenario)

print("Agent's trajectory:")
print("-" * 20)

# Display the conversation (system, user, assistant, tools)
messages = result_trajectory.messages()
for i, msg in enumerate(messages):
    role = msg.get("role", "unknown")
    content = msg.get("content", "")
    tool_calls = msg.get("tool_calls", [])

    if role == "system":
        print(
            f"[SYSTEM]: {content[:100]}..."
            if len(content) > 100
            else f"[SYSTEM]: {content}"
        )
    elif role == "user":
        print(f"[USER]: {content}")
    elif role == "assistant":
        if tool_calls:
            print(f"[ASSISTANT - TOOL CALLS]: {tool_calls}")
        if content:
            print(f"[ASSISTANT]: {content}")
    elif role == "tool":
        tool_name = msg.get("name", "unknown_tool")
        print(
            f"[TOOL - {tool_name}]: {content[:200]}..."
            if len(content) > 200
            else f"[TOOL - {tool_name}]: {content}"
        )

    print()

print("-" * 50)

# Print final answer and evaluation
if result_trajectory.final_answer:
    print(f"Agent's Final Answer:\n{result_trajectory.final_answer.answer}")
    print(f"Source Call IDs Used: {result_trajectory.final_answer.source_ids}")
else:
    print("No final answer provided by the agent.")

print(f"\nExpected Answer:\n{test_scenario.answer}")

# Optional: show judge scores if available
if result_trajectory.metrics:
    print("\nEvaluation metrics:")
    for k, v in result_trajectory.metrics.items():
        print(f"  {k}: {v}")

print("\n🎉 Call transcript agent testing completed!")
print(
    "The agent was evaluated using the same rollout and judge logic as during training."
)

Testing the trained model with a real call scenario...

Test scenario ID: 462
Question: What issue did the customer report with their checking account and how was it resolved?
Expected answer: The customer reported an unexpected overdraft fee on their checking account despite having a deposit that should have covered the balance. The agent investigated and found that the deposit was on hold for verification, which caused the overdraft. The agent explained the hold policy and removed the overdraft fee as a one-time courtesy.
--------------------------------------------------
Agent's trajectory:
--------------------
[SYSTEM]: 
You are a call transcript search agent.

You are given:
- A user question about banking issues (cre...

[USER]: What issue did the customer report with their checking account and how was it resolved?

[ASSISTANT - TOOL CALLS]: [{'id': 'chatcmpl-tool-f69e1b410c92459c8535eef855d52014', 'type': 'function', 'function': {'name': 'search_inbox', 'arguments': '{"keywords"

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord.png" height="50"></a>
<a href="https://art.openpipe.ai"><img src="https://github.com/openpipe/art/raw/main/assets/Documentation_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>
